### Python to SQL server connection and pushing the data

In [1]:
##### importing the libraries
import os
from distutils.util import strtobool
import pandas as pd
import pyodbc
from sqlalchemy import create_engine, update
import urllib
from sqlalchemy import MetaData,Table
from sqlalchemy.sql import text as txt

In [2]:
#### connection of python to SQL server using SQLalchemy configuration

def connect_to_sql_server(server_name, database_name):

    # Connect using sql alchemy
    driver_name = 'SQL Server Native Client 11.0'
    params = urllib.parse.quote_plus('''Driver={};
                                     Server={};
                                     Database={};
                                    Trusted_Connection=yes'''
                                    .format(driver_name, server_name,
                                             database_name))

    engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

    return engine

In [6]:
#### declaring configuration varaibles

server = 'AAPL-L-627\SQLEXPRESS'
database = 'test'
table = 'Table_2'
# pk = 'Col1'

In [7]:
### making the connection
engine = connect_to_sql_server(server, database)

In [ ]:
##### truncate the sql server table
def delete_from_table(database_name, table_name, replace_flag, engine):
    if replace_flag:
        truncate_str = "DELETE FROM " + database_name + "." + "dbo" + "." + table_name
        engine.execute(txt(truncate_str).execution_options(autocommit=True))
        print(f'Truncating table {table_name} because replace_flag is True')

In [ ]:
##### function to push the data from a dataframe to SQL server table

def push_to_sql(mod_df, table_name, primary_key_name,engine):

    tmp_var = 0
    counter = 0
    # Fetch all primary keys from the sql server
    query_str = """SELECT {primary_key_name}
    from {table_name}""".format(primary_key_name=primary_key_name,
    table_name=table_name)

    all_sql_df = pd.read_sql_query(query_str, engine)

    # Get all keys
    all_server_keys = list(all_sql_df[primary_key_name])

    # eg primary key is CHURN_KEY_ID
    # The dataframe which we passed is req_df
    new_records = mod_df[~mod_df[primary_key_name].isin(all_server_keys)].copy()

    if len(new_records) > 0:
        print(f"""{len(new_records)} new records to be 
              pushed to table {table_name}""")
        new_records.to_sql(table_name, con=engine,
                           if_exists='append', index=False)
        tmp_var = len(new_records)

    # Already existing records
    exist_records = mod_df[mod_df[primary_key_name].isin(all_server_keys)].copy()

    if len(exist_records) > 0:
        print(f"""{len(exist_records)} records 
              to be updated in the table {table_name}""")

        md = MetaData(engine)
        table = Table(table_name, md)

        where_str = table_name + "." + primary_key_name
        conn = engine.connect()
        
        # iterating the rows and updating the values 
        for index, row in exist_records.iterrows():
            filter_val = row[primary_key_name]
            tmp_dict = row.to_dict()
            tmp_dict = {txt(key):value for key,value in tmp_dict.items()}
            
            stmt = table.update().\
                    where(txt(where_str +" = " + "'" + filter_val +"'")).\
                    values(tmp_dict)
    
            conn.execute(stmt)
            counter = counter + 1

        conn.close()
    
    # Compute flags for both updation and insertion
    update_flag = bool(counter == len(exist_records))
    insert_flag = bool(tmp_var == len(new_records))
    
    return insert_flag, update_flag

In [ ]:
### calling the delete function
delete_from_table(database,table,True,engine)

In [ ]:
#### reading the data
data2 = pd.read_csv(r'D:\book1.csv')

In [ ]:
data2

In [ ]:
status = push_to_sql(data2,table,pk,engine)

In [ ]:
data3 = pd.read_csv(r'D:\book1.csv')

In [ ]:
data3

In [ ]:
status = push_to_sql(data3,table,pk,engine)

In [ ]:
data4 = pd.read_csv(r'D:\book1.csv')

In [ ]:
data5 = data4.head(2)

In [ ]:
data5

In [ ]:
status = push_to_sql(data5,table,pk,engine)

In [5]:
engine

Engine(mssql+pyodbc:///?odbc_connect=Driver%3DSQL+Server+Native+Client+11.0%3B%0A+++++++++++++++++++++++++++++++++++++Server%3DAAPL-L-627%5CSQLEXPRESS%3B%0A+++++++++++++++++++++++++++++++++++++Database%3Dtest1%3B%0A++++++++++++++++++++++++++++++++++++Trusted_Connection%3Dyes)